# A peak under the hood

In [7]:
# This example endows rectangle objects with color, to illustrate class variables.
    
# Modify the Rectangle class to print its instances nicely.
class Rectangle(object):
    """
    Represents a rectangle. Knows its width and height. Automatically updates its area
    in response to changes in its width or height.
    """
    def __init__(self, width, height):
        self._width  = 1.0  # We have to set this to a valid value for the setter on line 12 to work.
        self._height = 1.0  # We have to set this to a valid value for the setter on line 12 to work.
        self._area   = None
        
        self.width  = width 
        self.height = height
        
    # Here we choose a different kind of representation for Rectangle than we did for Vector.
    # We also incidentally point out that you don't have to hardcode the name of a class. A classn
    # can discover its own name at runtime.
    def __repr__(self):
        return "<{name}: width = {width}, height = {height}>".format(name = self.__class__.__name__,
                                                                           width = self.width, 
                                                                           height = self.height)
        
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, new_width):
        assert new_width > 0, "width must be greater than 0"
        self._width = new_width
        self._area  = self._calc_area()
        
    
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, new_height):
        assert new_height > 0, "height must be greater than 0"
        self._height = new_height
        self._area   = self._calc_area()
    
    @property
    def area(self):
        return self._area
    
    def _calc_area(self):
        return self.width * self.height
    

In [9]:


class ColoredRectangle(Rectangle):
    """
    Represents a colored rectangle. Knows its width, height. and color. Automatically updates its area
    in response to changes in its width or height. Disallows setting of non-positive width or height.
    Knows how to declare its color. All rectangles of the same subclass share the same color.
    """
    COLOR = 'CLEAR' # Default color for colored rectangles.
        
    # Define a method that prints out the color of a rectangle.
    def declare_color(self):
        """
        Print out a sentence that asserts the color we have.
        """
        print("My color is {color}".format(color = self.COLOR))
    
    
class BlueRectangle(ColoredRectangle):
    """
    The class of all blue rectangles
    """
    COLOR = 'BLUE' # All BlueRectangle instances are blue.
    
    
class RedRectangle(ColoredRectangle):
    """
    The class of all red rectangles
    """
    COLOR = 'RED' # All RedRectangle instances are red.
    
# Examples
rectangle = ColoredRectangle(3,4)
rectangle.declare_color()

blue_rectangle = BlueRectangle(1,2)
blue_rectangle.declare_color()

red_rectangle1 = RedRectangle(5,6)
red_rectangle2 = RedRectangle(7,8)
red_rectangle1.declare_color()
red_rectangle2.declare_color()

print("Now we are defying convention and globally changing the value of COLOR for red rectangles to CRIMSON")
RedRectangle.COLOR = 'CRIMSON' # This is an example only. In real life, we would not defy convention like this.

# Now the color of all instances of RedRectangle is CRIMSON
red_rectangle1.declare_color()
red_rectangle2.declare_color()

My color is CLEAR
My color is BLUE
My color is RED
My color is RED
Now we are defying convention and globally changing the value of COLOR for red rectangles to CRIMSON
My color is CRIMSON
My color is CRIMSON


Since the `declare_color()` method defined on line 12 above only references the class variable `COLOR`, one might think that the method could be called from a class object, instead of from an instance object, like this: `RedRectangle.declare_color()`. In fact, this does not work, as illustrated below.

In [1]:
# This does not work
try:
    RedRectangle.declare_color()
except Exception as e:
    print("Got the following error:", e)

Got the following error: name 'RedRectangle' is not defined


In fact, the error we get above makes perfect sense. `RedRectangle` is an object, so we are allowed to reference one of its attributes. `declare_color` is such an attribute, so `RedRectangle.declare_color` makes perfect sense, and refers to the function defined on line [36]:12 above. But that function takes one argument, called `self`, and we have not passed such an argument in line 3 above. If we _do_ call the function with an appropriate argument, then it works, as illustrated below:

In [2]:
# This works
RedRectangle.declare_color(RedRectangle)

NameError: name 'RedRectangle' is not defined

All of which might make us wonder why an expression like `red_rectangle2.declare_color()` works! Indeed, the magic is here, in understanding how python gets such an expression to do the right thing. We will not go into a detailed answer at the moment, but the short answer is that `RedRectangle.declare_color` is just a function, while `red_rectangle2.declare_color` is a _method_, as illustrated below:

In [3]:
# Referencing a function that has been defined on a class just returns that function as a normal function
print(RedRectangle.declare_color)

# Referencing the same function through an _instance_, and invoking the inheritance mechanism to access
# the attribute through its class parent, transforms the function into a *method*
print(red_rectangle2.declare_color)

NameError: name 'RedRectangle' is not defined

In line 2 above, we are simply making a function call and supplying the proper arguments to the function, so there is no mystery. As an aside, supplying an argument of `red_rectangle` would _also_ be supplying a proper argument, so this would also work:

In [4]:
RedRectangle.declare_color(red_rectangle2)

NameError: name 'RedRectangle' is not defined

When we define a method inside a class, we are really just defining a function, and it can always be called just as a function, if we know how to do it correctly. For example, this _also_ works: